# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Structured Streaming** </center>
---

**Date**: October, 2025

**Student Name**: Vicente Sebastian Serrano Cabrera

**Professor**: Pablo Camarillo Ramirez

python3 lib/Modulo_Sebas/producerValo.py kafka:9093 valorant-topic data/valorant_events.csv

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Structured Streaming (Project)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ff188128-ebdb-4ac4-b9cf-ccd8220a30ab;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

# Consumer

## Dataset and Stream creation

In [4]:
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "valorant-topic") \
            .load()

kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructField, StringType
from pcamarillor.spark_utils import SparkUtils
valo_df = kafka_df.select(kafka_df.value.cast("string").alias("value_str"))

schema_columns = [
    ("event_id", "string"),
    ("match_id", "string"),
    ("player_id", "string"),
    ("player_name", "string"),
    ("rank", "string"),
    ("map_name", "string"),
    ("game_mode", "string"),
    ("event_type", "string"),
    ("weapon_or_ability", "string"),
    ("timestamp", "string")
]

pkg_schema = SparkUtils.generate_schema(schema_columns)
valo_extracted_df = valo_df.withColumn("event_data", from_json(valo_df.value_str, pkg_schema))
valo_extracted_df.printSchema()            

root
 |-- value_str: string (nullable = true)
 |-- event_data: struct (nullable = true)
 |    |-- event_id: string (nullable = true)
 |    |-- match_id: string (nullable = true)
 |    |-- player_id: string (nullable = true)
 |    |-- player_name: string (nullable = true)
 |    |-- rank: string (nullable = true)
 |    |-- map_name: string (nullable = true)
 |    |-- game_mode: string (nullable = true)
 |    |-- event_type: string (nullable = true)
 |    |-- weapon_or_ability: string (nullable = true)
 |    |-- timestamp: string (nullable = true)

